In [1]:
import pickle
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from imblearn.under_sampling import EditedNearestNeighbours
import numpy as np
import seaborn as sns
import pandas as pd
from catboost import CatBoostClassifier

/home/jose/anaconda3/envs/py/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/jose/anaconda3/envs/py/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/jose/anaconda3/envs/py/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  depreca

In [2]:
data = pd.read_csv('/home/jose/Escritorio/datathon/src/data/train.txt', sep='|', index_col='ID')
# test = pd.read_csv('/home/jose/Escritorio/datathon/src/data/test.txt', sep='|', index_col='ID')

labels = data.iloc[:, -1].map({
    'RESIDENTIAL': -1,
    'INDUSTRIAL': -1,
    'PUBLIC': 1,
    'OFFICE': -1,
    'RETAIL': -1,
    'AGRICULTURE': -1,
    'OTHER': -1,
})
data.drop('CLASE', axis=1, inplace=True)

train, test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data = pd.concat([train, test], sort=False)

data['CADASTRALQUALITYID'] = data['CADASTRALQUALITYID'].map({'9': '0',
                               '8': '1',
                               '7': '2',
                               '6': '3',
                               '5': '4',
                               '4': '5',
                               '3': '6',
                               '2': '7',
                               '1': '8',
                               'C': '9',
                               'B': '10',
                               'A': '11',})
data['CADASTRALQUALITYID'] = data['CADASTRALQUALITYID'].astype('category')

In [3]:
data['MAXBUILDINGFLOOR'].fillna(data['MAXBUILDINGFLOOR'].median(), inplace=True)
data['CADASTRALQUALITYID'].fillna(data['CADASTRALQUALITYID'].mode()[0], inplace=True)

In [4]:
sc = StandardScaler()
data = pd.concat([pd.DataFrame(sc.fit_transform(data.select_dtypes(['number'])),
                              index=data.index,
                              columns=data.select_dtypes(['number']).columns),
                data.select_dtypes(['category', 'object'])], axis=1, sort=False)

train, test = data.iloc[:train.shape[0], ], data.iloc[train.shape[0]:, ]

data_number = data.copy()
data_number['CADASTRALQUALITYID'] = sc.fit_transform(data['CADASTRALQUALITYID'].astype(np.int).values.reshape(-1,1))

train_number, test_number = data_number.iloc[:train.shape[0], ], data_number.iloc[train.shape[0]:, ]

In [5]:
model = KNeighborsClassifier(n_jobs=-1)

model.fit(train_number, y_train)
y_pred = model.predict(test_number)

In [6]:
accuracy_score(y_test, y_pred)

0.97190739126223

In [7]:
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

          -1     0.9742    0.9975    0.9857     20048
           1     0.5763    0.1137    0.1899       598

    accuracy                         0.9719     20646
   macro avg     0.7752    0.5556    0.5878     20646
weighted avg     0.9627    0.9719    0.9627     20646



In [8]:
print(confusion_matrix(y_test, y_pred))

[[19998    50]
 [  530    68]]


In [9]:
params = False

if params:
    params = {'n_neighbors': [1,3,5,7,11,13,15,21],
             'weights': ['uniform', 'distance'],
             'metric': ['minkowski', 'manhattan']}

    model = KNeighborsClassifier()

    grid = GridSearchCV(model, params, cv=5, scoring=make_scorer(f1_score), n_jobs=-1)

    grid.fit(train_number, y_train)
    
    best_params = grid.best_params_
    
    model = KNeighborsClassifier(**best_params)
    model.fit(train_number, y_train)
    
    print(best_params)
else:
    best_params = {'metric': 'manhattan', 'n_jobs': -1, 'n_neighbors': 1, 'weights': 'uniform'}
    model = KNeighborsClassifier(**best_params)
    model.fit(train_number, y_train)

In [10]:
y_pred = model.predict(test_number)

In [11]:
accuracy_score(y_test, y_pred)

0.9635280441732055

In [12]:
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

          -1     0.9781    0.9844    0.9813     20048
           1     0.3348    0.2625    0.2943       598

    accuracy                         0.9635     20646
   macro avg     0.6564    0.6235    0.6378     20646
weighted avg     0.9595    0.9635    0.9614     20646



In [13]:
print(confusion_matrix(y_test, y_pred))

[[19736   312]
 [  441   157]]


In [14]:
params = True

if params:
    params = {'max_depth': [3,6,12,16,20],
             'learning_rate': np.linspace(0.01, 0.5, 6),
              'gamma': [0,2,5,10],
              'n_estimators': [400,800,1000,1200]}

    model = XGBClassifier(random_state=123)

    grid = RandomizedSearchCV(model, params, cv=5, scoring=make_scorer(f1_score), n_jobs=-1)

    grid.fit(train_number, y_train)
    
    best_params = grid.best_params_
    
    model = XGBClassifier(**best_params)
    model.fit(train_number, y_train)
    
    print(best_params)
else:
    best_params = {'n_jobs': -1, 'n_estimators': 1000, 'max_depth': 20}
    model = XGBClassifier(**best_params)
    model.fit(train_number, y_train)

/home/jose/anaconda3/envs/py/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


{'n_estimators': 400, 'max_depth': 20, 'learning_rate': 0.20600000000000002, 'gamma': 0}


In [15]:
y_pred = model.predict(test_number)

In [16]:
accuracy_score(y_test, y_pred)

0.9728276663760534

In [17]:
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

          -1     0.9753    0.9973    0.9862     20048
           1     0.6276    0.1522    0.2450       598

    accuracy                         0.9728     20646
   macro avg     0.8014    0.5747    0.6156     20646
weighted avg     0.9652    0.9728    0.9647     20646



In [18]:
print(confusion_matrix(y_test, y_pred))

[[19994    54]
 [  507    91]]


In [19]:
model = CatBoostClassifier(task_type='GPU', cat_features=[train.shape[1]-1])

model.fit(train, y_train)
y_pred = model.predict(test)

Learning rate set to 0.026445
0:	learn: 0.6439237	total: 12.7ms	remaining: 12.7s
1:	learn: 0.5978118	total: 25.6ms	remaining: 12.8s
2:	learn: 0.5561584	total: 43.5ms	remaining: 14.5s
3:	learn: 0.5177065	total: 55.2ms	remaining: 13.7s
4:	learn: 0.4840801	total: 65.5ms	remaining: 13s
5:	learn: 0.4534550	total: 76.7ms	remaining: 12.7s
6:	learn: 0.4235115	total: 87.5ms	remaining: 12.4s
7:	learn: 0.3974503	total: 98ms	remaining: 12.2s
8:	learn: 0.3732356	total: 110ms	remaining: 12.1s
9:	learn: 0.3512524	total: 121ms	remaining: 12s
10:	learn: 0.3331659	total: 133ms	remaining: 12s
11:	learn: 0.3161570	total: 145ms	remaining: 11.9s
12:	learn: 0.2983646	total: 157ms	remaining: 11.9s
13:	learn: 0.2853797	total: 168ms	remaining: 11.8s
14:	learn: 0.2729185	total: 179ms	remaining: 11.7s
15:	learn: 0.2614325	total: 188ms	remaining: 11.6s
16:	learn: 0.2486266	total: 198ms	remaining: 11.5s
17:	learn: 0.2386154	total: 209ms	remaining: 11.4s
18:	learn: 0.2285493	total: 219ms	remaining: 11.3s
19:	learn: 

175:	learn: 0.0948726	total: 1.84s	remaining: 8.61s
176:	learn: 0.0948350	total: 1.85s	remaining: 8.61s
177:	learn: 0.0947821	total: 1.87s	remaining: 8.64s
178:	learn: 0.0947551	total: 1.88s	remaining: 8.63s
179:	learn: 0.0947383	total: 1.89s	remaining: 8.63s
180:	learn: 0.0947134	total: 1.91s	remaining: 8.65s
181:	learn: 0.0946817	total: 1.92s	remaining: 8.64s
182:	learn: 0.0946248	total: 1.93s	remaining: 8.63s
183:	learn: 0.0945863	total: 1.94s	remaining: 8.62s
184:	learn: 0.0945281	total: 1.95s	remaining: 8.61s
185:	learn: 0.0944986	total: 1.96s	remaining: 8.6s
186:	learn: 0.0944587	total: 1.97s	remaining: 8.59s
187:	learn: 0.0944438	total: 1.99s	remaining: 8.58s
188:	learn: 0.0944091	total: 2s	remaining: 8.57s
189:	learn: 0.0943869	total: 2.01s	remaining: 8.56s
190:	learn: 0.0943370	total: 2.02s	remaining: 8.55s
191:	learn: 0.0943102	total: 2.03s	remaining: 8.54s
192:	learn: 0.0942844	total: 2.04s	remaining: 8.52s
193:	learn: 0.0942504	total: 2.05s	remaining: 8.51s
194:	learn: 0.09

342:	learn: 0.0903215	total: 3.67s	remaining: 7.04s
343:	learn: 0.0903043	total: 3.68s	remaining: 7.02s
344:	learn: 0.0902792	total: 3.69s	remaining: 7.01s
345:	learn: 0.0902690	total: 3.7s	remaining: 7s
346:	learn: 0.0902513	total: 3.71s	remaining: 6.99s
347:	learn: 0.0902392	total: 3.72s	remaining: 6.98s
348:	learn: 0.0902070	total: 3.73s	remaining: 6.97s
349:	learn: 0.0901921	total: 3.75s	remaining: 6.96s
350:	learn: 0.0901784	total: 3.76s	remaining: 6.95s
351:	learn: 0.0901605	total: 3.77s	remaining: 6.93s
352:	learn: 0.0901394	total: 3.78s	remaining: 6.92s
353:	learn: 0.0901246	total: 3.79s	remaining: 6.91s
354:	learn: 0.0901076	total: 3.8s	remaining: 6.9s
355:	learn: 0.0900963	total: 3.81s	remaining: 6.88s
356:	learn: 0.0900835	total: 3.82s	remaining: 6.87s
357:	learn: 0.0900667	total: 3.83s	remaining: 6.86s
358:	learn: 0.0900507	total: 3.84s	remaining: 6.86s
359:	learn: 0.0900296	total: 3.85s	remaining: 6.85s
360:	learn: 0.0900139	total: 3.86s	remaining: 6.84s
361:	learn: 0.0899

502:	learn: 0.0873542	total: 5.5s	remaining: 5.43s
503:	learn: 0.0873355	total: 5.51s	remaining: 5.42s
504:	learn: 0.0873271	total: 5.52s	remaining: 5.41s
505:	learn: 0.0873130	total: 5.53s	remaining: 5.4s
506:	learn: 0.0872920	total: 5.54s	remaining: 5.39s
507:	learn: 0.0872832	total: 5.55s	remaining: 5.38s
508:	learn: 0.0872698	total: 5.57s	remaining: 5.37s
509:	learn: 0.0872556	total: 5.58s	remaining: 5.36s
510:	learn: 0.0872410	total: 5.59s	remaining: 5.35s
511:	learn: 0.0872335	total: 5.6s	remaining: 5.34s
512:	learn: 0.0872246	total: 5.61s	remaining: 5.33s
513:	learn: 0.0872015	total: 5.62s	remaining: 5.32s
514:	learn: 0.0871797	total: 5.63s	remaining: 5.31s
515:	learn: 0.0871545	total: 5.65s	remaining: 5.3s
516:	learn: 0.0871399	total: 5.66s	remaining: 5.29s
517:	learn: 0.0871170	total: 5.67s	remaining: 5.28s
518:	learn: 0.0870944	total: 5.68s	remaining: 5.26s
519:	learn: 0.0870622	total: 5.69s	remaining: 5.25s
520:	learn: 0.0870497	total: 5.7s	remaining: 5.24s
521:	learn: 0.087

669:	learn: 0.0845248	total: 7.33s	remaining: 3.61s
670:	learn: 0.0845055	total: 7.34s	remaining: 3.6s
671:	learn: 0.0844860	total: 7.35s	remaining: 3.59s
672:	learn: 0.0844654	total: 7.36s	remaining: 3.57s
673:	learn: 0.0844481	total: 7.37s	remaining: 3.56s
674:	learn: 0.0844323	total: 7.38s	remaining: 3.55s
675:	learn: 0.0844266	total: 7.39s	remaining: 3.54s
676:	learn: 0.0844105	total: 7.39s	remaining: 3.53s
677:	learn: 0.0843952	total: 7.41s	remaining: 3.52s
678:	learn: 0.0843827	total: 7.42s	remaining: 3.51s
679:	learn: 0.0843668	total: 7.43s	remaining: 3.49s
680:	learn: 0.0843515	total: 7.44s	remaining: 3.48s
681:	learn: 0.0843336	total: 7.45s	remaining: 3.47s
682:	learn: 0.0843184	total: 7.46s	remaining: 3.46s
683:	learn: 0.0843011	total: 7.47s	remaining: 3.45s
684:	learn: 0.0842935	total: 7.48s	remaining: 3.44s
685:	learn: 0.0842785	total: 7.49s	remaining: 3.43s
686:	learn: 0.0842368	total: 7.5s	remaining: 3.42s
687:	learn: 0.0842256	total: 7.51s	remaining: 3.41s
688:	learn: 0.

841:	learn: 0.0820609	total: 9.18s	remaining: 1.72s
842:	learn: 0.0820527	total: 9.19s	remaining: 1.71s
843:	learn: 0.0820386	total: 9.2s	remaining: 1.7s
844:	learn: 0.0820305	total: 9.21s	remaining: 1.69s
845:	learn: 0.0820192	total: 9.22s	remaining: 1.68s
846:	learn: 0.0820002	total: 9.23s	remaining: 1.67s
847:	learn: 0.0819879	total: 9.24s	remaining: 1.66s
848:	learn: 0.0819822	total: 9.25s	remaining: 1.64s
849:	learn: 0.0819686	total: 9.26s	remaining: 1.63s
850:	learn: 0.0819627	total: 9.27s	remaining: 1.62s
851:	learn: 0.0819533	total: 9.28s	remaining: 1.61s
852:	learn: 0.0819430	total: 9.29s	remaining: 1.6s
853:	learn: 0.0819344	total: 9.3s	remaining: 1.59s
854:	learn: 0.0819237	total: 9.31s	remaining: 1.58s
855:	learn: 0.0819130	total: 9.32s	remaining: 1.57s
856:	learn: 0.0818960	total: 9.33s	remaining: 1.56s
857:	learn: 0.0818876	total: 9.34s	remaining: 1.55s
858:	learn: 0.0818751	total: 9.35s	remaining: 1.53s
859:	learn: 0.0818581	total: 9.37s	remaining: 1.52s
860:	learn: 0.08

CPU times: user 13.9 s, sys: 2.82 s, total: 16.7 s
Wall time: 11.6 s


In [20]:
accuracy_score(y_test, y_pred)

0.9729245374406664

In [21]:
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

          -1     0.9738    0.9990    0.9862     20048
           1     0.7407    0.1003    0.1767       598

    accuracy                         0.9729     20646
   macro avg     0.8573    0.5496    0.5815     20646
weighted avg     0.9671    0.9729    0.9628     20646



In [22]:
print(confusion_matrix(y_test, y_pred))

[[20027    21]
 [  538    60]]


In [ ]:
params = True

if params:
    aux = list(np.linspace(0.01, 0.5, 6))
    aux.append(None)
    params = {'iterations': [600,1000,1200,1500],
             'learning_rate': aux,
             'depth': [6, 8 ,16, 20]}

    model = CatBoostClassifier(task_type='GPU', cat_features=[train.shape[1]-1])

    grid = RandomizedSearchCV(model, params, cv=5, scoring=make_scorer(f1_score))

    grid.fit(train, y_train)
    
    best_params = grid.best_params_
    
    model = KNeighborsClassifier(**best_params)
    model.fit(train, y_train)
    
    print(best_params)
else:
    best_params = {'metric': 'manhattan', 'n_neighbors': 1, 'weights': 'uniform'}
    model = KNeighborsClassifier(**best_params)
    model.fit(train_number, y_train)

0:	learn: 0.2207424	total: 19.7ms	remaining: 23.6s
1:	learn: 0.1316447	total: 41ms	remaining: 24.5s
2:	learn: 0.1101438	total: 61.1ms	remaining: 24.4s
3:	learn: 0.1030587	total: 84.2ms	remaining: 25.2s
4:	learn: 0.1004518	total: 105ms	remaining: 25.1s
5:	learn: 0.0977807	total: 126ms	remaining: 25.1s
6:	learn: 0.0966109	total: 147ms	remaining: 25.1s
7:	learn: 0.0952672	total: 168ms	remaining: 25s
8:	learn: 0.0943907	total: 188ms	remaining: 24.8s
9:	learn: 0.0934888	total: 207ms	remaining: 24.7s
10:	learn: 0.0931415	total: 228ms	remaining: 24.7s
11:	learn: 0.0916992	total: 249ms	remaining: 24.7s
12:	learn: 0.0902609	total: 334ms	remaining: 30.5s
13:	learn: 0.0895484	total: 356ms	remaining: 30.2s
14:	learn: 0.0890521	total: 379ms	remaining: 29.9s
15:	learn: 0.0876987	total: 401ms	remaining: 29.7s
16:	learn: 0.0871381	total: 435ms	remaining: 30.3s
17:	learn: 0.0868294	total: 458ms	remaining: 30.1s
18:	learn: 0.0864316	total: 478ms	remaining: 29.7s
19:	learn: 0.0857685	total: 498ms	remaini

170:	learn: 0.0371474	total: 3.8s	remaining: 22.9s
171:	learn: 0.0370526	total: 3.83s	remaining: 22.9s
172:	learn: 0.0368693	total: 3.85s	remaining: 22.8s
173:	learn: 0.0367523	total: 3.87s	remaining: 22.8s
174:	learn: 0.0365467	total: 3.89s	remaining: 22.8s
175:	learn: 0.0363671	total: 3.92s	remaining: 22.8s
176:	learn: 0.0361436	total: 3.94s	remaining: 22.8s
177:	learn: 0.0359547	total: 3.96s	remaining: 22.7s
178:	learn: 0.0359227	total: 3.98s	remaining: 22.7s
179:	learn: 0.0357950	total: 4s	remaining: 22.7s
180:	learn: 0.0354219	total: 4.03s	remaining: 22.7s
181:	learn: 0.0352996	total: 4.05s	remaining: 22.6s
182:	learn: 0.0350966	total: 4.07s	remaining: 22.6s
183:	learn: 0.0349728	total: 4.09s	remaining: 22.6s
184:	learn: 0.0347334	total: 4.12s	remaining: 22.6s
185:	learn: 0.0346453	total: 4.14s	remaining: 22.5s
186:	learn: 0.0344602	total: 4.16s	remaining: 22.5s
187:	learn: 0.0343120	total: 4.18s	remaining: 22.5s
188:	learn: 0.0342247	total: 4.2s	remaining: 22.5s
189:	learn: 0.033

335:	learn: 0.0167538	total: 7.32s	remaining: 18.8s
336:	learn: 0.0167221	total: 7.34s	remaining: 18.8s
337:	learn: 0.0165799	total: 7.36s	remaining: 18.8s
338:	learn: 0.0165649	total: 7.38s	remaining: 18.7s
339:	learn: 0.0165239	total: 7.39s	remaining: 18.7s
340:	learn: 0.0163877	total: 7.42s	remaining: 18.7s
341:	learn: 0.0163336	total: 7.44s	remaining: 18.7s
342:	learn: 0.0162927	total: 7.46s	remaining: 18.6s
343:	learn: 0.0162507	total: 7.48s	remaining: 18.6s
344:	learn: 0.0162032	total: 7.52s	remaining: 18.6s
345:	learn: 0.0161257	total: 7.55s	remaining: 18.6s
346:	learn: 0.0160325	total: 7.57s	remaining: 18.6s
347:	learn: 0.0159657	total: 7.6s	remaining: 18.6s
348:	learn: 0.0159045	total: 7.62s	remaining: 18.6s
349:	learn: 0.0157544	total: 7.64s	remaining: 18.6s
350:	learn: 0.0156467	total: 7.67s	remaining: 18.5s
351:	learn: 0.0155422	total: 7.69s	remaining: 18.5s
352:	learn: 0.0154912	total: 7.71s	remaining: 18.5s
353:	learn: 0.0153964	total: 7.73s	remaining: 18.5s
354:	learn: 0

497:	learn: 0.0083152	total: 10.9s	remaining: 15.3s
498:	learn: 0.0082612	total: 10.9s	remaining: 15.3s
499:	learn: 0.0082202	total: 10.9s	remaining: 15.3s
500:	learn: 0.0081964	total: 10.9s	remaining: 15.3s
501:	learn: 0.0081732	total: 11s	remaining: 15.2s
502:	learn: 0.0081514	total: 11s	remaining: 15.2s
503:	learn: 0.0081124	total: 11s	remaining: 15.2s
504:	learn: 0.0080774	total: 11s	remaining: 15.2s
505:	learn: 0.0080622	total: 11s	remaining: 15.2s
506:	learn: 0.0080528	total: 11.1s	remaining: 15.1s
507:	learn: 0.0080397	total: 11.1s	remaining: 15.1s
508:	learn: 0.0080031	total: 11.1s	remaining: 15.1s
509:	learn: 0.0079710	total: 11.1s	remaining: 15.1s
510:	learn: 0.0079500	total: 11.2s	remaining: 15s
511:	learn: 0.0079246	total: 11.2s	remaining: 15s
512:	learn: 0.0079114	total: 11.2s	remaining: 15s
513:	learn: 0.0078920	total: 11.2s	remaining: 15s
514:	learn: 0.0078604	total: 11.3s	remaining: 15s
515:	learn: 0.0078275	total: 11.3s	remaining: 14.9s
516:	learn: 0.0077766	total: 11.

663:	learn: 0.0046468	total: 14.5s	remaining: 11.7s
664:	learn: 0.0046271	total: 14.5s	remaining: 11.7s
665:	learn: 0.0046150	total: 14.6s	remaining: 11.7s
666:	learn: 0.0045955	total: 14.6s	remaining: 11.6s
667:	learn: 0.0045789	total: 14.6s	remaining: 11.6s
668:	learn: 0.0045760	total: 14.6s	remaining: 11.6s
669:	learn: 0.0045648	total: 14.6s	remaining: 11.6s
670:	learn: 0.0045518	total: 14.7s	remaining: 11.6s
671:	learn: 0.0045352	total: 14.7s	remaining: 11.5s
672:	learn: 0.0045271	total: 14.7s	remaining: 11.5s
673:	learn: 0.0045160	total: 14.7s	remaining: 11.5s
674:	learn: 0.0045080	total: 14.7s	remaining: 11.5s
675:	learn: 0.0044905	total: 14.8s	remaining: 11.4s
676:	learn: 0.0044780	total: 14.8s	remaining: 11.4s
677:	learn: 0.0044680	total: 14.8s	remaining: 11.4s
678:	learn: 0.0044579	total: 14.8s	remaining: 11.4s
679:	learn: 0.0044480	total: 14.8s	remaining: 11.3s
680:	learn: 0.0044431	total: 14.9s	remaining: 11.3s
681:	learn: 0.0044327	total: 14.9s	remaining: 11.3s
682:	learn: 

...

63:	learn: 0.0091786	total: 21ms	remaining: 12.6s
64:	learn: 0.0089090	total: 21ms	remaining: 12.6s
65:	learn: 0.0086677	total: 21ms	remaining: 12.6s
66:	learn: 0.0084265	total: 21ms	remaining: 12.6s
67:	learn: 0.0083177	total: 21ms	remaining: 12.6s
68:	learn: 0.0079722	total: 21ms	remaining: 12.6s
69:	learn: 0.0077967	total: 21ms	remaining: 12.6s
70:	learn: 0.0075934	total: 21ms	remaining: 12.6s
71:	learn: 0.0075056	total: 21ms	remaining: 12.6s
72:	learn: 0.0072965	total: 21ms	remaining: 12.6s
73:	learn: 0.0071706	total: 21ms	remaining: 12.6s
74:	learn: 0.0070079	total: 21ms	remaining: 12.6s
75:	learn: 0.0068282	total: 21ms	remaining: 12.6s
76:	learn: 0.0067081	total: 21ms	remaining: 12.6s
77:	learn: 0.0065177	total: 21ms	remaining: 12.6s
78:	learn: 0.0063643	total: 21ms	remaining: 12.6s
79:	learn: 0.0062310	total: 21ms	remaining: 12.6s
80:	learn: 0.0061606	total: 21ms	remaining: 12.6s
81:	learn: 0.0059963	total: 21ms	remaining: 12.6s
82:	learn: 0.0058861	total: 21ms	remaining: 12.6s


225:	learn: 0.0013345	total: 21ms	remaining: 12.6s
226:	learn: 0.0013260	total: 21ms	remaining: 12.6s
227:	learn: 0.0013192	total: 21ms	remaining: 12.6s
228:	learn: 0.0013096	total: 21ms	remaining: 12.6s
229:	learn: 0.0013005	total: 21ms	remaining: 12.6s
230:	learn: 0.0012919	total: 21ms	remaining: 12.6s
231:	learn: 0.0012847	total: 21ms	remaining: 12.6s
232:	learn: 0.0012727	total: 21ms	remaining: 12.6s
233:	learn: 0.0012645	total: 21ms	remaining: 12.6s
234:	learn: 0.0012550	total: 21ms	remaining: 12.6s
235:	learn: 0.0012483	total: 21ms	remaining: 12.6s
236:	learn: 0.0012400	total: 21ms	remaining: 12.6s
237:	learn: 0.0012327	total: 21ms	remaining: 12.6s
238:	learn: 0.0012246	total: 21ms	remaining: 12.6s
239:	learn: 0.0012179	total: 21ms	remaining: 12.6s
240:	learn: 0.0012107	total: 21ms	remaining: 12.6s
241:	learn: 0.0012022	total: 21ms	remaining: 12.6s
242:	learn: 0.0011984	total: 21ms	remaining: 12.6s
243:	learn: 0.0011896	total: 21ms	remaining: 12.6s
244:	learn: 0.0011834	total: 21

386:	learn: 0.0006340	total: 21ms	remaining: 12.6s
387:	learn: 0.0006316	total: 21ms	remaining: 12.6s
388:	learn: 0.0006301	total: 21ms	remaining: 12.6s
389:	learn: 0.0006286	total: 21ms	remaining: 12.6s
390:	learn: 0.0006255	total: 21ms	remaining: 12.6s
391:	learn: 0.0006235	total: 21ms	remaining: 12.6s
392:	learn: 0.0006222	total: 21ms	remaining: 12.6s
393:	learn: 0.0006203	total: 21ms	remaining: 12.6s
394:	learn: 0.0006180	total: 21ms	remaining: 12.6s
395:	learn: 0.0006167	total: 21ms	remaining: 12.6s
396:	learn: 0.0006137	total: 21ms	remaining: 12.6s
397:	learn: 0.0006128	total: 21ms	remaining: 12.6s
398:	learn: 0.0006099	total: 21ms	remaining: 12.6s
399:	learn: 0.0006086	total: 21ms	remaining: 12.6s
400:	learn: 0.0006058	total: 21ms	remaining: 12.6s
401:	learn: 0.0006041	total: 21ms	remaining: 12.6s
402:	learn: 0.0006019	total: 21ms	remaining: 12.6s
403:	learn: 0.0006000	total: 21ms	remaining: 12.6s
404:	learn: 0.0005977	total: 21ms	remaining: 12.6s
405:	learn: 0.0005970	total: 21

547:	learn: 0.0004153	total: 21ms	remaining: 12.6s
548:	learn: 0.0004145	total: 21ms	remaining: 12.6s
549:	learn: 0.0004136	total: 21ms	remaining: 12.6s
550:	learn: 0.0004127	total: 21ms	remaining: 12.6s
551:	learn: 0.0004119	total: 21ms	remaining: 12.6s
552:	learn: 0.0004109	total: 21ms	remaining: 12.6s
553:	learn: 0.0004102	total: 21ms	remaining: 12.6s
554:	learn: 0.0004091	total: 21ms	remaining: 12.6s
555:	learn: 0.0004083	total: 21ms	remaining: 12.6s
556:	learn: 0.0004074	total: 21ms	remaining: 12.6s
557:	learn: 0.0004061	total: 21ms	remaining: 12.6s
558:	learn: 0.0004056	total: 21ms	remaining: 12.6s
559:	learn: 0.0004050	total: 21ms	remaining: 12.6s
560:	learn: 0.0004042	total: 21ms	remaining: 12.6s
561:	learn: 0.0004035	total: 21ms	remaining: 12.6s
562:	learn: 0.0004028	total: 21ms	remaining: 12.6s
563:	learn: 0.0004023	total: 21ms	remaining: 12.6s
564:	learn: 0.0004017	total: 21ms	remaining: 12.6s
565:	learn: 0.0004006	total: 21ms	remaining: 12.6s
566:	learn: 0.0003996	total: 21

104:	learn: 0.0040132	total: 5m 10s	remaining: 24m 22s
105:	learn: 0.0039794	total: 5m 13s	remaining: 24m 20s
106:	learn: 0.0039103	total: 5m 16s	remaining: 24m 19s
107:	learn: 0.0038313	total: 5m 20s	remaining: 24m 17s
108:	learn: 0.0037864	total: 5m 23s	remaining: 24m 16s
109:	learn: 0.0037484	total: 5m 26s	remaining: 24m 15s
110:	learn: 0.0036961	total: 5m 30s	remaining: 24m 14s
111:	learn: 0.0036448	total: 5m 33s	remaining: 24m 12s
112:	learn: 0.0036206	total: 5m 36s	remaining: 24m 10s
113:	learn: 0.0035635	total: 5m 39s	remaining: 24m 8s
114:	learn: 0.0035197	total: 5m 43s	remaining: 24m 6s
115:	learn: 0.0034583	total: 5m 46s	remaining: 24m 4s
116:	learn: 0.0034183	total: 5m 49s	remaining: 24m 2s
117:	learn: 0.0033432	total: 5m 52s	remaining: 24m 1s
118:	learn: 0.0033152	total: 5m 56s	remaining: 23m 59s
119:	learn: 0.0032617	total: 5m 59s	remaining: 23m 57s
120:	learn: 0.0032141	total: 6m 2s	remaining: 23m 55s
121:	learn: 0.0031712	total: 6m 5s	remaining: 23m 53s
122:	learn: 0.003

In [ ]:
y_pred = model.predict(test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
print(confusion_matrix(y_test, y_pred))